Atributos que serão coletados.

1.  date: Data da publicação do tweet
2.  name: Contém o nome da usuário que postou o tweet
3.  description: Contém a descrição do usuário do tweet
4.  location: Localização no qual foi postado o tweet
5.  text - Conteúdo postado do tweet

E-mail: giovannadomoreira@gmail.com

In [157]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

#remover stopwords
import nltk
from nltk.corpus import stopwords

#remover expressoes regulares
import re
import emoji


# Aquisição.

Para aquisição dos dados foi feito a busca pela palavras boticário e os nomes das marcas da perfumario do Boticário. Algumas marcas tem um nome muito comum o que faz que a raspagem de tweets busque tweets fora do contexto do projeto, por exemplo a linha Coffee, pensando nisso para burlar esse problema utilizarei a palavra perfume na frente das palavras com nomes comuns. 

In [131]:
keywords =['boticario','Malbec','perfume Alchemist','The Blend','perfume Zaad','perfume Quasar','Egeo','Dr. Botica','perfume Sophie',
           'perfume Capricho','Nativa Spa','Floratta','perfume Glamour','perfume Lily','BotiCollection','Elysée','perfume Celebre','perfume Liz',
           'perfume Connexion','perfume Coffee','perfume Linda','perfume Men','Accordes','Arbo','Portinari','Styletto','Uomini','perfume Insensatez','perfume Dream','Botica 214',
           'Boti Baby','perfume Make B.']

tweets_list = []
    
for word in keywords:
    maxTweets = 5000
    query = f'{word} lang:pt since:2022-06-06 until:2022-06-19'
    
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        tweets_list.append([tweet.date, tweet.likeCount, tweet.user.username,tweet.user.location, tweet.content])
        if len(tweets_list) >= maxTweets:
            break

df = pd.DataFrame(tweets_list, columns=['date', 'likes', 'user','location', 'text'])
df
    

C:\Users\abraz\AppData\Local\Temp\ipykernel_12504\2878185119.py:13: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets_list.append([tweet.date, tweet.likeCount, tweet.user.username,tweet.user.location, tweet.content])
Skipping unrecognised entry ID: 'promoted-tweet-1660653353016205316-2ca4da05f408d176'
Skipping unrecognised entry ID: 'promoted-tweet-1661020659626000384-2ca4bfbbce708d5c'
Skipping unrecognised entry ID: 'promoted-tweet-1524760007358783488-2ca4da4d99c521a4'
Skipping unrecognised entry ID: 'promoted-tweet-1660653353016205316-2ca4c377af44a9bc'
Skipping unrecognised entry ID: 'promoted-tweet-1661020659626000384-2ca4b3f74c2a32fe'
Skipping unrecognised entry ID: 'promoted-tweet-1524760007358783488-2ca445f4637c6628'
Skipping unrecognised entry ID: 'promoted-tweet-1659302844066045958-2ca4e3a794f28640'
Skipping unrecognised entry ID: 'promoted-tweet-1661020659626000384-2ca4854782a433a2'
Skipping unrecognised entry ID: 'promoted-tweet-1524760007358783

,date,likes,user,location,text
0,2022-06-18 23:52:57+00:00,0,kejkjm7,Tocantins e Paraná,Gente eu amo os perfumes egeo da boticário
1,2022-06-18 23:52:40+00:00,1,Kevlinha,,"Carai botica 214 da boticário eu te amo ""dark ..."
2,2022-06-18 23:49:25+00:00,0,BruChristina_,"Rio de Janeiro, Brasil",Comprar o Boticário com a minha tia é um perig...
3,2022-06-18 23:44:35+00:00,0,christina12567,,@CentralEslo Boticário\nESLU EM CARUARU
4,2022-06-18 23:41:23+00:00,1,BarcellossClara,"Nilópolis, Brasil",Uso natura e boticário feliz da vida kkkkkkkkk
...,...,...,...,...,...
5006,2022-06-14 20:33:02+00:00,1,PromobitOficial,Brasil,#promoção #oferta [Parcelado] Desodorante Col...
5007,2022-06-18 21:51:08+00:00,2,zani_greta,"Milano, Lombardia",o studiar e diventar uomini o volar a Empoli p...
5008,2022-06-18 04:25:26+00:00,0,imnootlim,court of dreams,@nosoymadu O PERFUME DE DROGA!!
5009,2022-06-18 23:52:40+00:00,1,Kevlinha,,"Carai botica 214 da boticário eu te amo ""dark ..."


In [135]:
df.text.loc[657]

'Boticário com mil lançamentos e zero brinde pra nós :('

In [136]:
#salvando df
df.to_csv('tweets_raw.csv', index=False)

# Processar.

Foram coletado 5011 tweets com base nas palavras de busca, para dar inicio ao pré-processamento, irei fazer uma cópia da coluna text do df em um novo arquivo data frame para fazer o controle e o comparativo das etapas do processamento do texto.

In [252]:
df_text = pd.read_csv('tweets_raw.csv')
df_text = pd.DataFrame(df_text['text'].copy())
df_text.head()

,text
0,Gente eu amo os perfumes egeo da boticário
1,"Carai botica 214 da boticário eu te amo ""dark ..."
2,Comprar o Boticário com a minha tia é um perig...
3,@CentralEslo Boticário\nESLU EM CARUARU
4,Uso natura e boticário feliz da vida kkkkkkkkk


## Deixando tudo em letra minúcula.

Uma técnica muito comundo no processamento de texto é normalizar as letras para caixa baixa para que palavras iguais não sejam entendidas pelo modelo diferentes, exemplo: Perfume e perfume.

In [253]:
df_text['text_lower'] = df_text['text'].str.lower()
df_text.head()

,text,text_lower
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ..."
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk


## Removendo usuario marcados com @

Irei fazer a remoção de nome de úsuario pois tais informações não ao objeivo do projeto.

In [254]:
#localziadno textos que tem @
df_urls = df_text[df_text['text_lower'].str.contains('@', na = False)]
df_urls.head()

,text,text_lower
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru
7,@paulo_capeletti Entra no site da Boticário aí...,@paulo_capeletti entra no site da boticário aí...
8,O creme deleite da @boticario é o melhor ❤️,o creme deleite da @boticario é o melhor ❤️
9,"@Antonio09636317 Amigo, tô contigo.\nMês passa...","@antonio09636317 amigo, tô contigo.\nmês passa..."
16,@LulaOficial Ela vende o quê? Jequiti ou Botic...,@lulaoficial ela vende o quê? jequiti ou botic...


In [255]:
def remove_user(text):
    #Procura texto com o padrão especificado
    url_path = re.compile(r'@\S+')
    #substitui
    text_s_url = re.sub(url_path,'', text)
    return text_s_url

#testenado a funçao
text = df_text.text_lower.loc[8]
print(f'Texto original: {text}')
print(f'Texto sem url: {remove_user(text)}')

Texto original: o creme deleite da @boticario é o melhor ❤️
Texto sem url: o creme deleite da  é o melhor ❤️


In [256]:
df_text['text_w_user'] = df_text['text_lower'].apply(remove_user)
df_text.head()

,text,text_lower,text_w_user
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ..."
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk


## Removendo pontuação.

Ainda na sessão de remoção de valores que não agregam ao objetivo do projeto irei retirar todas as pontuações.

In [258]:
pont = '[!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~]'
df_text['text_pont'] = df_text['text_w_user'].str.replace(pont,'')
df_text.head()

C:\Users\abraz\AppData\Local\Temp\ipykernel_3148\1024300077.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_text['text_pont'] = df_text['text_w_user'].str.replace(pont,'')


,text,text_lower,text_w_user,text_pont
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...",carai botica 214 da boticário eu te amo dark m...
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário\neslu em caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk


In [261]:
df_text['text_pont'].loc[1000]

'fui na boticário hoje e tinha um moço com três sacolas da santa lola e tava comprando mais presentes lá né aí fiquei me perguntando se todos os presentes eram pra uma única moça ou se era um pra cada molier \U0001fae3'

## Removendo quebra de linhas.

In [264]:
df_text['text_pont'] = df_text['text_pont'].str.replace('\n',' ')
df_text.head()

,text,text_lower,text_w_user,text_pont
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...",carai botica 214 da boticário eu te amo dark m...
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk


## Removendo stopwords

Irei fazer a remoção das stop words. Para aqueles que não estão familiarizados com o conceito,nas palavras do www.clubedoportugues.com.br/stop-words/ "Stop word significa a palavra que pode ser totalmente suprimida, omitida ou ocultada na hora de fazer uma busca na internet sem que o sentido do que se quer encontrar seja perdido. Isso acontece, porque simplesmente as funções semânticas dessas palavras não se mostram tão determinantes para garantir resultados eficazes nos buscadores".

In [223]:

def remove_stopwords(text, lingua='portuguese'):
    '''func para remover stopwords'''
    stop_words = set(stopwords.words(lingua))
    words = text.split()
    words_sem_sw = [word for word in words if word not in stop_words]
    return ' '.join(words_sem_sw)

df_text['text_sw'] = df_text['text_pont'].apply(remove_stopwords)
df_text.head()


,text,text_lower,text_w_user,text_w_htag,text_pont,text_sw
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente amo perfumes egeo boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...",carai botica 214 da boticário eu te amo dark m...,carai botica 214 boticário amo dark mint pqp s...
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar boticário tia perigo sai parcelando pe...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru,boticário eslu caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura boticário feliz vida kkkkkkkkk


## Removendo URLS

In [224]:
#localziadno textos que tem urls
df_urls = df_text[df_text['text_sw'].str.contains('https', na = False)]
df_urls.head()

,text,text_lower,text_w_user,text_w_htag,text_pont,text_sw
15,Pow depois de comprar uma bebida com gosto de ...,pow depois de comprar uma bebida com gosto de ...,pow depois de comprar uma bebida com gosto de ...,pow depois de comprar uma bebida com gosto de ...,pow depois de comprar uma bebida com gosto de ...,pow comprar bebida gosto floratta boticário pr...
23,Lembra? Boticário maquiou mulheres no dia do d...,lembra? boticário maquiou mulheres no dia do d...,lembra? boticário maquiou mulheres no dia do d...,lembra? boticário maquiou mulheres no dia do d...,lembra boticário maquiou mulheres no dia do di...,lembra boticário maquiou mulheres dia divórcio...
34,😱😱😱😱😱😱\n\nColônia Quasar Next Star Wars The Ma...,😱😱😱😱😱😱\n\ncolônia quasar next star wars the ma...,😱😱😱😱😱😱\n\ncolônia quasar next star wars the ma...,😱😱😱😱😱😱\n\ncolônia quasar next star wars the ma...,😱😱😱😱😱😱 colônia quasar next star wars the mand...,😱😱😱😱😱😱 colônia quasar next star wars the manda...
35,😱😱😱😱😱😱\n\nDesodorante Colônia L'eau de Lily 75...,😱😱😱😱😱😱\n\ndesodorante colônia l'eau de lily 75...,😱😱😱😱😱😱\n\ndesodorante colônia l'eau de lily 75...,😱😱😱😱😱😱\n\ndesodorante colônia l'eau de lily 75...,😱😱😱😱😱😱 desodorante colônia leau de lily 75ml ...,😱😱😱😱😱😱 desodorante colônia leau lily 75ml boti...
41,Jackie Pinheiro: Rebecca se une ao Boticário p...,jackie pinheiro: rebecca se une ao boticário p...,jackie pinheiro: rebecca se une ao boticário p...,jackie pinheiro: rebecca se une ao boticário p...,jackie pinheiro rebecca se une ao boticário pa...,jackie pinheiro rebecca une boticário lançar n...


In [225]:
def remove_url(text):
    #Procura texto com o padrão especificado
    url_path = re.compile(r'https?\S+|www\S+')
    #substitui
    text_s_url = re.sub(url_path, '', text)
    return text_s_url

#testenado a funçao
text = df_text.text_sw.loc[15]
print(f'Texto original: {text}')
print(f'Texto sem url: {remove_url(text)}')

Texto original: pow comprar bebida gosto floratta boticário provei aqui boa viu podem ir gosto rexona v8 httpstcovvi0f7gh6e
Texto sem url: pow comprar bebida gosto floratta boticário provei aqui boa viu podem ir gosto rexona v8 


In [226]:
df_text['text_w_urls'] = df_text['text_sw'].apply(remove_url)
df_text.head()

,text,text_lower,text_w_user,text_w_htag,text_pont,text_sw,text_w_urls
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...",carai botica 214 da boticário eu te amo dark m...,carai botica 214 boticário amo dark mint pqp s...,carai botica 214 boticário amo dark mint pqp s...
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru,boticário eslu caruaru,boticário eslu caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura boticário feliz vida kkkkkkkkk,uso natura boticário feliz vida kkkkkkkkk


## Convertendo emojis.


Por mais que os emojis sejam de fato texto, para o objetivo do projeto esses carinhas carregam muito contexo e carga sentimental e por isso optei em fazer a conversão dos emojis para palavras.

In [227]:
#identificando emojis
emoji_pattern = re.compile(r'[^\w\s,]')
df_urls = df_text[df_text['text_sw'].str.contains(emoji_pattern, na = False)]
df_urls.head()

,text,text_lower,text_w_user,text_w_htag,text_pont,text_sw,text_w_urls
5,"Ainda mais que quase ngm tem meu cheiro, impor...","ainda mais que quase ngm tem meu cheiro, impor...","ainda mais que quase ngm tem meu cheiro, impor...","ainda mais que quase ngm tem meu cheiro, impor...",ainda mais que quase ngm tem meu cheiro import...,ainda quase ngm cheiro importadissimo kkkkk mo...,ainda quase ngm cheiro importadissimo kkkkk mo...
8,O creme deleite da @boticario é o melhor ❤️,o creme deleite da @boticario é o melhor ❤️,o creme deleite da é o melhor ❤️,o creme deleite da é o melhor ❤️,o creme deleite da é o melhor ❤️,creme deleite melhor ❤️,creme deleite melhor ❤️
10,Comprei 2 perfumes p mim na boticário. 😍,comprei 2 perfumes p mim na boticário. 😍,comprei 2 perfumes p mim na boticário. 😍,comprei 2 perfumes p mim na boticário. 😍,comprei 2 perfumes p mim na boticário 😍,comprei 2 perfumes p mim boticário 😍,comprei 2 perfumes p mim boticário 😍
12,Meus amigos minha irmã me deu um perfume da Ca...,meus amigos minha irmã me deu um perfume da ca...,meus amigos minha irmã me deu um perfume da ca...,meus amigos minha irmã me deu um perfume da ca...,meus amigos minha irmã me deu um perfume da ca...,amigos irmã deu perfume carolina herrera prime...,amigos irmã deu perfume carolina herrera prime...
13,"Com sono, com fome, mas muito cheirosa obg lin...","com sono, com fome, mas muito cheirosa obg lin...","com sono, com fome, mas muito cheirosa obg lin...","com sono, com fome, mas muito cheirosa obg lin...",com sono com fome mas muito cheirosa obg linha...,sono fome cheirosa obg linha bumbum boticario 🤝,sono fome cheirosa obg linha bumbum boticario 🤝


In [228]:
#localizando
teste = df_text['text_sw'].loc[3000]
teste

'tô nojo uniforme inter malbec novo 😜🥱🥱'

In [229]:
#testando
def add_spaces_between_emojis(text):
    new_text = ''
    for c in text:
        if c in emoji.EMOJI_DATA:
            new_text += ' ' + c
        else:
            new_text += c
    return new_text.strip()


def convert_emoji(text): 
    emoji_name = emoji.demojize(text, language='pt').replace(",","").replace(":","")
    return emoji_name

convert_emoji(add_spaces_between_emojis(teste))

'tô nojo uniforme inter malbec novo  rosto_piscando_e_com_língua_para_fora rosto_bocejando rosto_bocejando'

In [245]:
#aplicando
df_text['text_w_emoji'] = df_text['text_w_urls'].apply(add_spaces_between_emojis)
df_text['text_w_emoji'] = df_text['text_w_emoji'].apply(convert_emoji)
df_text.head()

,text,text_lower,text_w_user,text_w_htag,text_pont,text_sw,text_w_urls,text_w_emoji,text_spell
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...",carai botica 214 da boticário eu te amo dark m...,carai botica 214 boticário amo dark mint pqp s...,carai botica 214 boticário amo dark mint pqp s...,carai botica 214 boticário amo dark mint pqp s...,carai botica 214 boticário amo dark mint pqp s...
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru,boticário eslu caruaru,boticário eslu caruaru,boticário eslu caruaru,boticário eslu caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura boticário feliz vida kkkkkkkkk,uso natura boticário feliz vida kkkkkkkkk,uso natura boticário feliz vida kkkkkkkkk,uso natura boticário feliz vida kkkkkkkkk


## Corrigindo a ortografia.

Utilizei a biblioteca spellcheker para fazer uma correção ortografica para que as palvras escritas de forma errada não sejam entendidas como palavras diferrentes.

In [246]:
from spellchecker import SpellChecker

spell = SpellChecker(language='pt')
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
text = "vocse connsegue corigiir ezta frase?"
correct_spellings(text)

'voce consegue corrigir esta frase'

In [247]:
df_text['text_spell'] = df_text['text_w_emoji'].apply(convert_emoji)
df_text.head()

,text,text_lower,text_w_user,text_w_htag,text_pont,text_sw,text_w_urls,text_w_emoji,text_spell
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...",carai botica 214 da boticário eu te amo dark m...,carai botica 214 boticário amo dark mint pqp s...,carai botica 214 boticário amo dark mint pqp s...,carai botica 214 boticário amo dark mint pqp s...,carai botica 214 boticário amo dark mint pqp s...
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru,boticário eslu caruaru,boticário eslu caruaru,boticário eslu caruaru,boticário eslu caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura boticário feliz vida kkkkkkkkk,uso natura boticário feliz vida kkkkkkkkk,uso natura boticário feliz vida kkkkkkkkk,uso natura boticário feliz vida kkkkkkkkk


# Df final.

Depois de todo processamento do dataframe chegamos ao conjunto final.

In [248]:
colunas = []
df_tweets_tratados = pd.DataFrame(columns=colunas)
df_tweets_tratados['text'] = df_text['text_spell']
df_tweets_tratados.tail()

,text
5006,parcelado desodorante colônia boticollection s...
5007,studiar diventar uomini volar empoli per il co...
5008,perfume droga
5009,carai botica 214 boticário amo dark mint pqp s...
5010,miguel perguntou queria alguma coisa aproveite...


In [263]:
#exemplo de tweet processado.

print(f'Original: {df_text.text.loc[1000]}')
print(f'Tratados: {df_tweets_tratados.text.loc[1000]}')

Original: fui na Boticário hoje e tinha um moço com três sacolas da santa Lola e tava comprando mais presentes lá né, aí fiquei me perguntando se todos os presentes eram pra uma única moça ou se era um pra cada molier 🫣
Tratados: boticário hoje moço três sacolas santa lola tava comprando presentes lá né aí fiquei perguntando todos presentes pra única moça pra cada molier  rosto_com_olho_espiando
